# Preprocesamiento de Datos
Limpieza, tokenización y vectorización de los textos.

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import joblib
import os
import spacy
from tqdm import tqdm
tqdm.pandas()

# Descargar recursos NLTK
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

# Cargar modelo de spaCy
nlp = spacy.load('es_core_news_sm')

# --- 1. Cargar dataset unificado ---
df = pd.read_csv('../data/processed/dataset_unificado.csv')
print(f"Dataset cargado: {len(df)} filas")
print(f"Distribución: {df['label'].value_counts().to_dict()}")

# --- 2. Preprocesamiento avanzado ---
def preprocesar_texto_avanzado(texto):
    if pd.isna(texto) or texto == "":
        return ""
    # Normalizar y limpiar
    texto = str(texto).lower()
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)
    texto = re.sub(r'[^a-záéíóúñü\s]', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    # Procesar con spaCy
    doc = nlp(texto)
    # Lematizar, eliminar stopwords y tokens cortos
    tokens = [
        token.lemma_ for token in doc
        if token.is_alpha and not token.is_stop and len(token.lemma_) > 2
    ]
    return ' '.join(tokens)

# --- 3. Aplicar preprocesamiento ---
print("🔄 Preprocesando textos...")
df['texto_procesado'] = df['contenido'].progress_apply(preprocesar_texto_avanzado)

# --- 4. Vectorización TF-IDF optimizada ---
print("🔄 Vectorizando con TF-IDF...")
vectorizer = TfidfVectorizer(
    max_features=3000,  # Reducido para evitar overfitting
    min_df=2,          # Palabra debe aparecer en al menos 2 documentos
    max_df=0.95,       # Palabra no debe aparecer en más del 95% de documentos
    ngram_range=(1, 2), # Unigramas y bigramas
    stop_words=None    # Ya eliminamos stopwords
)

X = vectorizer.fit_transform(df['texto_procesado'])
y = df['label'].map({'fake': 0, 'real': 1})

print(f"Matriz TF-IDF: {X.shape}")
print(f"Vocabulario: {len(vectorizer.vocabulary_)} palabras")

# --- 5. Split estratificado ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {X_train.shape[0]} | Test: {X_test.shape[0]}")

# --- 6. Guardar archivos ---
os.makedirs('../data/split', exist_ok=True)
os.makedirs('../models', exist_ok=True)

# Guardar datos procesados
joblib.dump((X_train, y_train), '../data/split/train.pkl')
joblib.dump((X_test, y_test), '../data/split/test.pkl')

# Guardar vectorizador
joblib.dump(vectorizer, '../models/vectorizer.pkl')

# Guardar dataset procesado
df_procesado = df[['contenido', 'texto_procesado', 'label']]
df_procesado.to_csv('../data/processed/dataset_procesado.csv', index=False)

print("✅ Preprocesamiento completado!")
print("�� Archivos guardados:")
print("  - /data/split/train.pkl")
print("  - /data/split/test.pkl") 
print("  - /models/vectorizer.pkl")
print("  - /data/processed/dataset_procesado.csv")

Dataset cargado: 1072 filas
Distribución: {'fake': 536, 'real': 536}
🔄 Preprocesando textos...


100%|█████████████████████████████████████████████████████████████████████| 1072/1072 [01:23<00:00, 12.85it/s]


🔄 Vectorizando con TF-IDF...
Matriz TF-IDF: (1072, 3000)
Vocabulario: 3000 palabras
Train: 857 | Test: 215
✅ Preprocesamiento completado!
�� Archivos guardados:
  - /data/split/train.pkl
  - /data/split/test.pkl
  - /models/vectorizer.pkl
  - /data/processed/dataset_procesado.csv
